#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

## Insert your ID number ("numero di matricola") below

In [46]:
# put here your ``numero di matricola''
numero_di_matricola = 1


In [47]:
# import all packages needed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline


Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [48]:
# load the data
df = pd.read_csv('kc_house_data.csv', sep=',')

# remove the data samples with missing values (NaN)
df = df.dropna()

df.describe()


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [49]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:", m)
Y = Data[:m, 2]
X = Data[:m, 3:]


Amount of data: 21613


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [50]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
from sklearn.model_selection import train_test_split
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:", m_train)
print("Amount of data for validation (choosing among different models):", m_val)
print("Amount of data for test:", m_test)

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(
    X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(
    Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)


Amount of data for training and deciding parameters: 14408
Amount of data for validation (choosing among different models): 3602
Amount of data for test: 3603


Let's standardize the data.

In [51]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)


## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [52]:
# let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

# let's define the model
# the default model performs extremely bad : R^2 negative...
# so we use the LBFGS solver
NN = MLPRegressor(hidden_layer_sizes=(1,), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# let's print the coefficients of the model for the input nodes (but not the bias)
print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# let's print the coefficient for the bias (i.e., the bias)
print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)


training error 0.29559863217335836
validation error 0.31059586274997
coeffs (weights from input layer to the hidden neuron):
 [array([[ -729.87486122],
       [  831.2303795 ],
       [ 1799.75148514],
       [  124.60013752],
       [  119.38146152],
       [ 1146.72653943],
       [  929.24974225],
       [  381.66546344],
       [ 2763.39475395],
       [ 1668.15762936],
       [  601.24288021],
       [-1820.67409758],
       [  126.75146032],
       [ -703.0951815 ],
       [ 1943.23571365],
       [ -759.15710926],
       [  300.88988146],
       [ -202.38632292]]), array([[43.04451409]])]
intercepts (weights of biases of input and hidden layers):
 [array([12602.10199491]), array([-5370.82907044])]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [53]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-LR.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-LR.score(Xval_scaled, Yval))

# let's print the coefficients of the model for the input nodes (but not the bias)
print("coef:\n", LR.coef_)

# let's print the coefficient for the bias (i.e., the bias)
print("intercept:\n", LR.intercept_)

training error 0.2971718751921173
validation error 0.3113977005845858
coef:
 [-32024.37205507  30857.38702503  81520.08274701   4358.71015115
   4210.76613832  52271.26763491  39275.36300358  17128.54179271
 114582.56984907  76295.51474423  25805.04611065 -75175.13990886
   6580.79073948 -29421.4439202   84347.82703947 -28791.00545764
   9512.22430408  -8875.3212928 ]
intercept:
 539147.4730705542


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [54]:
# let's write the code to learn a linear model with NN: how?

# let's define the model
# (notice, by default MLPRegressor uses squared loss, exactly like the linear model regressor)
NN = MLPRegressor(hidden_layer_sizes=(1,), random_state=numero_di_matricola, solver="lbfgs", activation="identity")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# let's print the coefficients of the model for the input nodes (but not the bias)
print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# let's print the coefficient for the bias (i.e., the bias)
print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)

# notice intercepts[0]*coefs[1,0]+intercepts[1] = intercept or linear model regressor

training error 0.2971718758498938
validation error 0.3113975786278589
coeffs (weights from input layer to the hidden neuron):
 [array([[  53.05892257],
       [ -51.11646483],
       [-135.33248854],
       [  -7.22545107],
       [  -6.98193883],
       [ -86.59959823],
       [ -65.07184335],
       [ -28.37751298],
       [-189.83995551],
       [-126.15950406],
       [ -42.62168198],
       [ 124.55003826],
       [ -10.9016803 ],
       [  48.75330118],
       [-139.74756357],
       [  47.70252967],
       [ -15.76187867],
       [  14.7062653 ]]), array([[-603.573508]])]
intercepts (weights of biases of input and hidden layers):
 [array([-892.60243245]), array([402.44197475])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [55]:
# COMPLETE


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [56]:
# let's build a NN with 2 nodes in the only hidden layer

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(2,), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# let's print the coefficients of the model for the input nodes (but not the bias)
print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# let's print the coefficient for the bias (i.e., the bias)
print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)



training error 0.2153208979874328
validation error 0.21690093519558962
coeffs (weights from input layer to the hidden neuron):
 [array([[ -32.10729555,  -16.38981375],
       [ 116.45979582,   27.15117946],
       [ 171.25183209,   55.99693335],
       [-288.5583707 ,   17.21660766],
       [ -98.12489751,   28.17114291],
       [ 106.85686761,   14.07110269],
       [ -29.1029425 ,   52.61406787],
       [  61.74518617,   26.7848754 ],
       [ 277.04724345,  136.12529807],
       [ 181.99576479,   49.44585152],
       [  13.99059296,   24.08211091],
       [ -53.63301901,  -90.12891609],
       [  28.99586503,    9.77813247],
       [-198.64003826,  -28.10630235],
       [ 296.80982024,  120.20746084],
       [-454.94082664,  -18.0556324 ],
       [  12.28997402,   62.54149262],
       [-320.04383237,   -5.6423691 ]]), array([[748.23410896],
       [656.7949309 ]])]
intercepts (weights of biases of input and hidden layers):
 [array([-1084.99886264,   778.08863079]), array([782.893831

Let's build a NN with 5 nodes in the only hidden layer

In [57]:
# let's build a NN with 5 nodes in the only hidden layer

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(5,), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)

training error 0.16409610419710907
validation error 0.17530164058251185


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [58]:
# let's build a NN with 10 nodes in the only hidden layer

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(10,), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)


training error 0.127399261512813
validation error 0.14331396707337007


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [59]:
# let's build a NN with 100 nodes in the only hidden layer

# let's define the model
NN = MLPRegressor(max_iter=1000, hidden_layer_sizes=(100,), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)

training error 0.05750129141969351
validation error 0.11642854289390092


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [60]:
# let's build a NN with 2 hidden layers, 1 node each

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(1,1), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)

training error 0.2709866028192074
validation error 0.28542758123222955


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 2 nodes each

In [61]:
# let's build a NN with 2 layers, 2 nodes each

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(2,2), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)

training error 0.24579106567172326
validation error 0.2516283049275865


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 10 nodes each

In [62]:
# let's build a NN with 2 layers, 10 nodes each

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(10,10), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)


training error 0.1490405350069579
validation error 0.16497647245517488


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [63]:
# let's build a NN with 2 layers, 100 nodes each

# let's define the model
NN = MLPRegressor(hidden_layer_sizes=(100,100), random_state=numero_di_matricola, solver="lbfgs")

# let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

# let's print the error (1 - R^2) on training data
print("training error", 1-NN.score(Xtrain_scaled, Ytrain))

# let's print the error (1 - R^2) on validation data
print("validation error", 1-NN.score(Xval_scaled, Yval))

# # let's print the coefficients of the model for the input nodes (but not the bias)
# print("coeffs (weights from input layer to the hidden neuron):\n", NN.coefs_)

# # let's print the coefficient for the bias (i.e., the bias)
# print("intercepts (weights of biases of input and hidden layers):\n", NN.intercepts_)


training error 0.06478762206269884
validation error 0.11640398000193142


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [64]:
from sklearn.model_selection import GridSearchCV

# COMPLETE


Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [65]:
# let's print the best model according to grid search
# COMPLETE

# let's print the error 1-R^2 for the best model
# COMPLETE


Let compare the error of the best NN on train and validation and on test data.

In [66]:
# COMPLETE


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [67]:
# COMPLETE


Note: MLPRegressor has several other parameters!